# Gliederung Prompt Engineering
### Starte mit einem generischen Prompt
### Füge eine professionelle Rolle hinzu
### Führe konkrete Einschränkungen ein
### Bonus: Wende an, was du gelernt hast

In [36]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables and initialize OpenAI client
load_dotenv()

# Azure OpenAI Configuration - following the pattern you showed
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# Model name should match your Azure deployment name
model = azure_deployment if azure_deployment else "gpt-4o-mini"

if not api_key or not azure_endpoint:
    raise ValueError("Azure OpenAI configuration missing. Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, and AZURE_OPENAI_DEPLOYMENT_NAME in your .env file.")

# Configure client for Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model=model):
    """Simple wrapper for OpenAI API calls."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [37]:
# No changes needed in this cell
from enum import Enum


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = "gpt-4.1-mini"

# COMPLETION FUNCTION
def get_completion(system_prompt, user_prompt, model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt
        user_prompt: The user prompt
        model: The model to use (default is gpt-4.1-mini)
    Returns:
        The completion text
    """
    messages = [
        {"role": "user", "content": user_prompt},
    ]
    if system_prompt is not None:
        messages = [
            {"role": "system", "content": system_prompt},
            *messages,
        ]
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

# RESPONSE DISPLAY FUNCTION
def display_responses(*args):
    """Helper function to display responses as Markdown, horizontally."""
    markdown_string = "<table><tr>"
    # Headers
    for arg in args:
        markdown_string += f"<th>System Prompt:<br />{arg['system_prompt']}<br /><br />"
        markdown_string += f"User Prompt:<br />{arg['user_prompt']}</th>"
    markdown_string += "</tr>"
    # Rows
    markdown_string += "<tr>"
    for arg in args:
        markdown_string += f"<td>Response:<br />{arg['response']}</td>"
    markdown_string += "</tr></table>"
    display(Markdown(markdown_string))

## 1. Generic Prompt

First, let's see what the model produces with a basic prompt.

In [38]:
# No changes needed in this cell
plain_system_prompt = "Du hilft Probleme zu lösen."  # A generic system prompt
user_prompt = "Mein Computer startet nicht mehr. Was kann ich tun?"

print(f"Sending prompt to {MODEL} model...")
baseline_response = get_completion(plain_system_prompt, user_prompt)
print("Response received!\n")

display_responses(
    {
        "system_prompt": plain_system_prompt,
        "user_prompt": user_prompt,
        "response": baseline_response,
    }
)

Sending prompt to gpt-4.1-mini model...
Response received!



<table><tr><th>System Prompt:<br />Du hilft Probleme zu lösen.<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th></tr><tr><td>Response:<br />Es tut mir leid zu hören, dass dein Computer nicht mehr startet. Hier sind einige Schritte, die du ausprobieren kannst, um das Problem zu beheben:

1. **Stromversorgung überprüfen**  
   - Stelle sicher, dass das Netzkabel fest mit dem Computer und der Steckdose verbunden ist.  
   - Prüfe, ob die Steckdose funktioniert, indem du ein anderes Gerät anschließt.  
   - Wenn du einen Laptop benutzt, überprüfe, ob der Akku geladen ist oder versuche, den Laptop ohne Akku nur mit Netzteil zu starten.

2. **Periphere Geräte entfernen**  
   - Trenne alle externen Geräte (USB-Sticks, Drucker, externe Festplatten etc.), da diese manchmal den Startvorgang behindern können.

3. **Hardware-Reset durchführen**  
   - Schalte den Computer komplett aus und trenne ihn vom Stromnetz.  
   - Halte den Power-Button ca. 15-30 Sekunden gedrückt, um Reststrom zu entladen.  
   - Schließe alles wieder an und versuche neu zu starten.

4. **Fehlermeldungen beachten**  
   - Gibt es Pieptöne oder blinkende LEDs beim Start? Falls ja, notiere die Anzahl und Muster der Signale, sie können Hinweise auf das Problem geben.

5. **Monitor überprüfen**  
   - Stelle sicher, dass der Monitor eingeschaltet und richtig angeschlossen ist.  
   - Probiere, den PC an einen anderen Monitor oder Fernseher anzuschließen, um auszuschließen, dass der Bildschirm defekt ist.

6. **BIOS/UEFI Zugriff versuchen**  
   - Versuche beim Start die BIOS/UEFI-Einstellungen zu öffnen (meist mit Tasten wie F2, DEL oder ESC). Wenn das gelingt, liegt das Problem möglicherweise am Betriebssystem.

7. **Boot-Medium prüfen**  
   - Wenn du kürzlich Änderungen an der Hardware vorgenommen hast, kontrolliere, ob die Festplatte korrekt erkannt wird.  
   - Eventuell kannst du mit einem bootfähigen USB-Stick oder einer Rettungs-CD deinen Computer starten und Fehler beheben.

8. **Technischen Support kontaktieren**  
   - Wenn die oben genannten Schritte nicht helfen, könnte ein Hardwaredefekt vorliegen (z.B. Netzteil, Mainboard, RAM). In diesem Fall ist es sinnvoll, einen Fachmann zu konsultieren.

Wenn du möchtest, kann ich dich auch durch spezifischere Schritte führen. Lass mich wissen, welche Art von Computer du hast (Laptop/PC), was genau passiert beim Einschalten (z.B. keine Reaktion, nur Lüfter drehen, Fehlermeldungen etc.).</td></tr></table>

## 2. Add a Professional Role

Now, let's add a professional role to see how it affects the response.

<div style="color: red">NOTE: We will use the same user prompt for these examples. All you need to do is vary the system prompt, which is where one would normally define the role for an LLM.</div>

In [39]:
# TODO: Write a system prompt starting with "You are..." replacing the ***********
role_system_prompt = """
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
"""


print("Sending prompt with professional role...")
role_response = get_completion(role_system_prompt, user_prompt)
print("Response received!\n")

# Show last two prompts and responses
display_responses(
    {
        "system_prompt": plain_system_prompt,
        "user_prompt": user_prompt,
        "response": baseline_response,
    },
    {
        "system_prompt": role_system_prompt,
        "user_prompt": user_prompt,
        "response": role_response,
    },
)

Sending prompt with professional role...


Response received!



<table><tr><th>System Prompt:<br />Du hilft Probleme zu lösen.<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th><th>System Prompt:<br />
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th></tr><tr><td>Response:<br />Es tut mir leid zu hören, dass dein Computer nicht mehr startet. Hier sind einige Schritte, die du ausprobieren kannst, um das Problem zu beheben:

1. **Stromversorgung überprüfen**  
   - Stelle sicher, dass das Netzkabel fest mit dem Computer und der Steckdose verbunden ist.  
   - Prüfe, ob die Steckdose funktioniert, indem du ein anderes Gerät anschließt.  
   - Wenn du einen Laptop benutzt, überprüfe, ob der Akku geladen ist oder versuche, den Laptop ohne Akku nur mit Netzteil zu starten.

2. **Periphere Geräte entfernen**  
   - Trenne alle externen Geräte (USB-Sticks, Drucker, externe Festplatten etc.), da diese manchmal den Startvorgang behindern können.

3. **Hardware-Reset durchführen**  
   - Schalte den Computer komplett aus und trenne ihn vom Stromnetz.  
   - Halte den Power-Button ca. 15-30 Sekunden gedrückt, um Reststrom zu entladen.  
   - Schließe alles wieder an und versuche neu zu starten.

4. **Fehlermeldungen beachten**  
   - Gibt es Pieptöne oder blinkende LEDs beim Start? Falls ja, notiere die Anzahl und Muster der Signale, sie können Hinweise auf das Problem geben.

5. **Monitor überprüfen**  
   - Stelle sicher, dass der Monitor eingeschaltet und richtig angeschlossen ist.  
   - Probiere, den PC an einen anderen Monitor oder Fernseher anzuschließen, um auszuschließen, dass der Bildschirm defekt ist.

6. **BIOS/UEFI Zugriff versuchen**  
   - Versuche beim Start die BIOS/UEFI-Einstellungen zu öffnen (meist mit Tasten wie F2, DEL oder ESC). Wenn das gelingt, liegt das Problem möglicherweise am Betriebssystem.

7. **Boot-Medium prüfen**  
   - Wenn du kürzlich Änderungen an der Hardware vorgenommen hast, kontrolliere, ob die Festplatte korrekt erkannt wird.  
   - Eventuell kannst du mit einem bootfähigen USB-Stick oder einer Rettungs-CD deinen Computer starten und Fehler beheben.

8. **Technischen Support kontaktieren**  
   - Wenn die oben genannten Schritte nicht helfen, könnte ein Hardwaredefekt vorliegen (z.B. Netzteil, Mainboard, RAM). In diesem Fall ist es sinnvoll, einen Fachmann zu konsultieren.

Wenn du möchtest, kann ich dich auch durch spezifischere Schritte führen. Lass mich wissen, welche Art von Computer du hast (Laptop/PC), was genau passiert beim Einschalten (z.B. keine Reaktion, nur Lüfter drehen, Fehlermeldungen etc.).</td><td>Response:<br />Da der Computer nicht mehr startet, sollten wir systematisch vorgehen, um die Ursache zu finden. Hier sind die ersten Schritte zur Analyse:

1. **Stromversorgung prüfen:**
   - Ist der Rechner überhaupt mit Strom versorgt? (Netzkabel, Steckdose, Netzschalter)
   - Funktioniert die Steckdose? (z.B. mit einem anderen Gerät testen)
   - Sind alle Netzteilkabel korrekt angeschlossen?

2. **Hardware-Selbsttest beachten:**
   - Gibt es irgendwelche LED-Anzeigen oder Pieptöne beim Einschalten?
   - Reagiert der Lüfter oder die Festplatte?

3. **Periphere Geräte abklemmen:**
   - Entfernen Sie alle nicht notwendigen Geräte (USB-Sticks, externe Festplatten, Drucker) und versuchen Sie erneut zu starten.

4. **BIOS/UEFI-Zugang prüfen:**
   - Kann man ins BIOS/UEFI gelangen? (meist über Entf, F2 o.ä. beim Start)

5. **Fehlermeldungen oder POST-Codes:**
   - Gibt es Fehlermeldungen oder blinkende LEDs, die auf einen bestimmten Fehler hinweisen?

**Mögliche Ursachen:**
- Netzteil defekt
- Mainboard- oder CPU-Problem
- RAM-Fehler
- Defekte Festplatte/SSD
- BIOS-Fehler oder Konfigurationsproblem

**Empfehlung:**
- Starten Sie mit der Stromversorgungskontrolle.
- Versuchen Sie, ins BIOS zu gelangen.
- Testen Sie, ob durch Entfernen von RAM-Modulen oder Peripherie der Startvorgang beeinflusst wird.
- Wenn keine Reaktion erfolgt, prüfen Sie das Netzteil mit einem Multimeter oder tauschen es testweise aus.
- Falls möglich, testen Sie den Rechner mit minimaler Hardware (Motherboard, CPU, ein RAM-Riegel, Stromversorgung).

Falls Sie nach diesen Schritten weitere Informationen haben, können wir die Diagnose weiter eingrenzen.</td></tr></table>

## 3. Ausgabeformat zur Weiterverarbeitung

Damit wir den Output des Agents gut parsen und weiterverarbeiten können, bitten wir ihn den Output im JSON FORMAT auszugeben.

In [40]:
# 3. Ausgabeformat zur Weiterverarbeitung
formated_system_prompt = f""" {role_system_prompt}. 
# Output Format
Bitte gebe deine Antwort im folgenden JSON Format. Gebe nur das JSON Objekt zurück, ohne weitere Erklärungen oder Text.
FORMAT:
{{
  "Ursache": "<mögliche Ursache des Problems>",
  "Empfehlung": "<konkrete Handlungsempfehlung zur Problemlösung>"
}}
EXAMPLE:
Hier ist ein Beispiel:
Input: "Mein Computer startet nicht mehr. Was kann ich tun?"
Output:
{{
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers oder eines Softwareproblems nicht starten.",
  "Empfehlung": "Überprüfen Sie die Stromversorgung und alle Kabelverbindungen. Versuchen Sie, den Computer im abgesicherten Modus zu starten. Wenn das Problem weiterhin besteht, führen Sie eine Systemwiederherstellung durch oder kontaktieren Sie den technischen Support."
}}
"""

print("Sending prompt with constraints...")
formated_response = get_completion(formated_system_prompt, user_prompt)
print("Response received!\n")

# Show last two prompts and responses
display_responses(
    {
        "system_prompt": role_system_prompt,
        "user_prompt": user_prompt,
        "response": role_response,
    },
    {
        "system_prompt": formated_system_prompt,
        "user_prompt": user_prompt,
        "response": formated_response,
    },
)

Sending prompt with constraints...
Response received!



<table><tr><th>System Prompt:<br />
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th><th>System Prompt:<br /> 
# ROLE
Du bist ein erfahrener IT-Supportmitarbeiter.
# TASK:
Du analysierst das Problem und identifizierst die mögliche Ursache.
Danach gibst du eine Empfehlung, wie das Problem gelöst werden kann.
# CONTEXT
Die User sind ebenfalls erfahrene IT-Mitarbeiter in einem mittelständischen Unternehmen.
. 
# Output Format
Bitte gebe deine Antwort im folgenden JSON Format. Gebe nur das JSON Objekt zurück, ohne weitere Erklärungen oder Text.
FORMAT:
{
  "Ursache": "<mögliche Ursache des Problems>",
  "Empfehlung": "<konkrete Handlungsempfehlung zur Problemlösung>"
}
EXAMPLE:
Hier ist ein Beispiel:
Input: "Mein Computer startet nicht mehr. Was kann ich tun?"
Output:
{
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers oder eines Softwareproblems nicht starten.",
  "Empfehlung": "Überprüfen Sie die Stromversorgung und alle Kabelverbindungen. Versuchen Sie, den Computer im abgesicherten Modus zu starten. Wenn das Problem weiterhin besteht, führen Sie eine Systemwiederherstellung durch oder kontaktieren Sie den technischen Support."
}
<br /><br />User Prompt:<br />Mein Computer startet nicht mehr. Was kann ich tun?</th></tr><tr><td>Response:<br />Da der Computer nicht mehr startet, sollten wir systematisch vorgehen, um die Ursache zu finden. Hier sind die ersten Schritte zur Analyse:

1. **Stromversorgung prüfen:**
   - Ist der Rechner überhaupt mit Strom versorgt? (Netzkabel, Steckdose, Netzschalter)
   - Funktioniert die Steckdose? (z.B. mit einem anderen Gerät testen)
   - Sind alle Netzteilkabel korrekt angeschlossen?

2. **Hardware-Selbsttest beachten:**
   - Gibt es irgendwelche LED-Anzeigen oder Pieptöne beim Einschalten?
   - Reagiert der Lüfter oder die Festplatte?

3. **Periphere Geräte abklemmen:**
   - Entfernen Sie alle nicht notwendigen Geräte (USB-Sticks, externe Festplatten, Drucker) und versuchen Sie erneut zu starten.

4. **BIOS/UEFI-Zugang prüfen:**
   - Kann man ins BIOS/UEFI gelangen? (meist über Entf, F2 o.ä. beim Start)

5. **Fehlermeldungen oder POST-Codes:**
   - Gibt es Fehlermeldungen oder blinkende LEDs, die auf einen bestimmten Fehler hinweisen?

**Mögliche Ursachen:**
- Netzteil defekt
- Mainboard- oder CPU-Problem
- RAM-Fehler
- Defekte Festplatte/SSD
- BIOS-Fehler oder Konfigurationsproblem

**Empfehlung:**
- Starten Sie mit der Stromversorgungskontrolle.
- Versuchen Sie, ins BIOS zu gelangen.
- Testen Sie, ob durch Entfernen von RAM-Modulen oder Peripherie der Startvorgang beeinflusst wird.
- Wenn keine Reaktion erfolgt, prüfen Sie das Netzteil mit einem Multimeter oder tauschen es testweise aus.
- Falls möglich, testen Sie den Rechner mit minimaler Hardware (Motherboard, CPU, ein RAM-Riegel, Stromversorgung).

Falls Sie nach diesen Schritten weitere Informationen haben, können wir die Diagnose weiter eingrenzen.</td><td>Response:<br />{
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers (z. B. Netzteil, Festplatte, RAM) oder eines Softwareproblems (z. B. beschädigtes Betriebssystem) nicht starten.",
  "Empfehlung": "Prüfen Sie zunächst die Stromversorgung und Kabelverbindungen. Starten Sie den Computer im BIOS/UEFI, um grundlegende Hardwareerkennung zu überprüfen. Testen Sie den RAM mit einem Diagnosetool und führen Sie gegebenenfalls eine Reparaturinstallation oder Systemwiederherstellung des Betriebssystems durch. Wenn möglich, verwenden Sie ein bootfähiges Medium zur weiteren Diagnose."
}</td></tr></table>

## LLM as a Judge

Wir nutzen nun ein weiteres LLM um den Output zu überprüfen.

In [41]:
judge_system_prompt = """
  Du bist ein sorgfältiger und detailorientierter Evaluator.
  Deine Aufgabe ist es, die Antworten eines anderen KI-Modells
   anhand folgender Kriterien zu bewerten:

  1. **Korrektheit**: Sind die bereitgestellten Informationen 
  richtig und relevant für das Problem des Benutzers?
  2. **Klarheit**: Ist die Antwort klar und leicht 
  verständlich?
  3. **Nützlichkeit**: Bietet die Antwort umsetzbare
  Ratschläge, die der Benutzer zur Lösung seines Problems
  befolgen kann?
  4. **Format-Konformität**: Entspricht die Antwort dem
  angeforderten JSON-Format ohne zusätzlichen Text oder
  Erklärungen?

  Bewerte jeden Aspekt mit einer Punktzahl von 1 bis 10 und       
  gib eine kurze Begründung.
  Strukturiere deine Bewertung übersichtlich und abschließend     
  mit einer Gesamtbewertung.

  Format deiner Antwort:
  - Verwende klare Überschriften
  - Strukturiere mit Aufzählungen
  - Gib am Ende eine Gesamtbewertung und Zusammenfassung
  """

  # Verwende die formatierte Antwort aus der vorherigen Zelle zur Bewertung
judge_user_prompt = f"""
  Bitte bewerte die folgende Antwort:

  {formated_response}

  Die ursprüngliche Benutzeranfrage war: "{user_prompt}"
  """
print("Sende an LLM-Richter zur Bewertung...")
judge_response = get_completion(judge_system_prompt,
judge_user_prompt)
print("Bewertung erhalten!\n")

# Zeige mit display_responses Funktion
display_responses(
      {
          "system_prompt": judge_system_prompt,
          "user_prompt": judge_user_prompt,
          "response": judge_response,
      }
  )


Sende an LLM-Richter zur Bewertung...
Bewertung erhalten!



<table><tr><th>System Prompt:<br />
  Du bist ein sorgfältiger und detailorientierter Evaluator.
  Deine Aufgabe ist es, die Antworten eines anderen KI-Modells
   anhand folgender Kriterien zu bewerten:

  1. **Korrektheit**: Sind die bereitgestellten Informationen 
  richtig und relevant für das Problem des Benutzers?
  2. **Klarheit**: Ist die Antwort klar und leicht 
  verständlich?
  3. **Nützlichkeit**: Bietet die Antwort umsetzbare
  Ratschläge, die der Benutzer zur Lösung seines Problems
  befolgen kann?
  4. **Format-Konformität**: Entspricht die Antwort dem
  angeforderten JSON-Format ohne zusätzlichen Text oder
  Erklärungen?

  Bewerte jeden Aspekt mit einer Punktzahl von 1 bis 10 und       
  gib eine kurze Begründung.
  Strukturiere deine Bewertung übersichtlich und abschließend     
  mit einer Gesamtbewertung.

  Format deiner Antwort:
  - Verwende klare Überschriften
  - Strukturiere mit Aufzählungen
  - Gib am Ende eine Gesamtbewertung und Zusammenfassung
  <br /><br />User Prompt:<br />
  Bitte bewerte die folgende Antwort:

  {
  "Ursache": "Der Computer könnte aufgrund eines Hardwarefehlers (z. B. Netzteil, Festplatte, RAM) oder eines Softwareproblems (z. B. beschädigtes Betriebssystem) nicht starten.",
  "Empfehlung": "Prüfen Sie zunächst die Stromversorgung und Kabelverbindungen. Starten Sie den Computer im BIOS/UEFI, um grundlegende Hardwareerkennung zu überprüfen. Testen Sie den RAM mit einem Diagnosetool und führen Sie gegebenenfalls eine Reparaturinstallation oder Systemwiederherstellung des Betriebssystems durch. Wenn möglich, verwenden Sie ein bootfähiges Medium zur weiteren Diagnose."
}

  Die ursprüngliche Benutzeranfrage war: "Mein Computer startet nicht mehr. Was kann ich tun?"
  </th></tr><tr><td>Response:<br />{
  "Korrektheit": 9,
  "Begründung_Korrektheit": "Die Antwort nennt korrekte und relevante Ursachen für das Startproblem des Computers sowie passende diagnostische Schritte und Lösungen.",
  "Klarheit": 8,
  "Begründung_Klarheit": "Die Antwort ist größtenteils klar formuliert, könnte aber durch eine noch einfachere Sprache und Strukturierung besser verständlich sein.",
  "Nützlichkeit": 9,
  "Begründung_Nützlichkeit": "Die gegebenen Empfehlungen sind praktisch und umsetzbar, bieten jedoch keine Priorisierung der Schritte, was besonders für Laien hilfreich wäre.",
  "Format-Konformität": 10,
  "Begründung_Format": "Die Antwort ist vollständig im angeforderten JSON-Format und enthält keine zusätzlichen Erklärungen oder Text.",
  "Gesamtbewertung": 9,
  "Zusammenfassung": "Die Antwort ist korrekt, nützlich und gut formatiert. Eine noch klarere und strukturiertere Darstellung mit Priorisierung der Maßnahmen würde die Qualität weiter steigern."
}</td></tr></table>